<a href="https://colab.research.google.com/github/sasuraibito1125/google_colab/blob/main/t%E5%88%86%E5%B8%83%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 関数定義

In [ ]:
# @title #### 四捨五入関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名: `round_up`
#@markdown * 引数：`precision=4`, `digit=None`
#@markdown * 戻り値：lambda function
def round_up(precision=4, digit=None):
  '''Generate lambda function to round off.

  ex)
  df.map(round_up(digit='0.0001'))
  df.map(round_up(digit='1E1'))

  Parameters
  ----------
  precision: precision of rounding. default is 4.
  digit: string of rounding digit. default is None(no used) and prioritize this to precision.

  Returns
  -------
  lambda function
    lambda function to round off.
  '''
  from decimal import Decimal, ROUND_HALF_UP
  def to_digit(precision):
    if precision == 0:
      return '0'
    else:
      return '0.' + ('0' * (precision - 1)) + '1'
  return lambda x: float(Decimal(str(x))
                            .quantize(Decimal(digit if digit else to_digit(precision)),
                                      rounding=ROUND_HALF_UP))

In [ ]:
#@title #### `DataFrame` から指定された数値のカラムとインデックスを見つける関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`find_number_values`
#@markdown * 引数：`df`, `v`, `closest=False`
#@markdown * 戻り値：`list<tuple<int, string, number, Series>>`
def find_number_values(df, v, closest=False):
  '''Find index and column of value matched or closest.

  Parameters
  ----------
  df: DataFrame to be searched.
  v: value to search for.
  closest: flag value to find closest value. default is False.

  Returns
  -------
  list
    list of tuples of index, column name, matched value, row(Series).
  '''
  # prepare
  df_nums = df.select_dtypes(include='number')
  monotonic = df_nums.apply(lambda s: s.is_monotonic_increasing or s.is_monotonic_decreasing).all()
  columns = df_nums.columns
  if closest:
    df_nums = (df_nums - v).abs()
    target = df_nums.min().min()
  else:
    target = v
  # print('TARGET:', target)  # DEBUG

  return [(idx, c, row[c], df.loc[idx])
            for (idx, row) in df_nums.iterrows() if row.min() <= target <= row.max()
            for c in columns if row[c] == target]


In [ ]:
# @title #### t分布表生成関数 { vertical-output: true, display-mode: "form" }
# @markdown 関数名：`generate_td_table`
# @markdown * 引数：`alpha=None`, `v=None`, `both=False`, `precision=4`
# @markdown * 戻り値：`DataFrame`
# @markdown * 依存関数：`round_up`
def generate_td_table(alpha=None, v=None, both=False, precision=4):
  '''To generate the table of t distribution.

  Parameters
  ----------
  alpha: list of points of upper. if not defined, values are [0.25, 0.1, 0.05, 0.025, 0.01, 0.005]
  v: list of degrees of freedom. if not defined, values are 1 to 30 and 40 to 100 per 10 steps
  both: flag to specify one-sided or both-sided. default False(both-sided).
  precision: precision of the value of probability. default 4.

  Returns
  -------
  DataFrame
    table of t distribution
  '''
  import numpy as np
  import pandas as pd
  from scipy.stats import t

  def generate_default_v():
    v1 = np.arange(1, 31, dtype=int)
    v2 = np.linspace(40, 100, 7, dtype=int)
    return np.concatenate((v1, v2)).reshape(-1, 1)

  if not alpha:
    alpha = [0.25, 0.1, 0.05, 0.025, 0.01, 0.005]
  alpha = np.array(alpha)

  if not v:
    v = generate_default_v()
  else:
    v = np.array(v)
    if v.ndim == 1:
      v = v.reshape(-1, 1)
    elif v.ndim == 2 and v.shape[1] == 1:
      pass
    else:
      print('不正な自由度のリストが指定されました。デフォルト値を使用します。')
      v = generate_default_v()

  df = pd.DataFrame(t.isf(alpha, v),
                    columns=list(map(lambda x: "%.3f" % x, alpha)),
                    index=list(map(lambda x: "%d" % x, v)))
  return (df * (2 if both else 1)).applymap(round_up(precision=precision))


In [ ]:
#@title #### 表の行と列を強調する関数
#@markdown 関数名：`highlight_subject`
#@markdown * 引数：`df`, `row=None`, `col=None`, `row_color='#FFD0FF'`, `col_color='#D1FDFF'`, `cross_color='#C7B5FF'`
#@markdown * 戻り値：`Styler`
def highlight_subject(df, row=None, col=None,
                      row_color='#FFD0FF',
                      col_color='#D1FDFF',
                      cross_color='#C7B5FF'):
  '''highlight row, column and cross point of DataFrame.

  Parameters
  ----------
  df: a target pandas DataFrame to be highlighted
  row: a target row to be highlighted
  col: a target column to be highlighted
  row_color: a highlight color of row
  col_color: a highlight color of column
  cross_color: a highlight color of cross point of highligh row and column

  Returns
  -------
  Styler
    pandas Styler object to be highlighted
  '''
  import pandas as pd

  idx = pd.IndexSlice
  styler = df.style
  if row:
    styler = styler.set_properties(**{'color':'black', 'background-color': row_color},
                                   subset=idx[idx[row], :])
  if col:
    styler = styler.set_properties(**{'color':'black', 'background-color': col_color},
                                   subset=idx[col])
  if col and row:
    styler = styler.set_properties(**{'color':'black', 'background-color': cross_color},
                                   subset=idx[idx[row], idx[col]])
  return styler


In [ ]:
#@title #### t分布表から指定された $t$ 値に最も近い $t$ 値に対する自由度と $\alpha$ を取得する関数
#@markdown 関数名：`find_alpha_n_from_t_in_td_table`
#@markdown * 引数：`td`, `t`
#@markdown * 戻り値：`tuple<string, string, number>`
#@markdown * 依存関数：`find_number_values`
def find_alpha_n_from_t_in_td_table(td, t):
  '''Find degree of freedom and alpha from the closest t-value in t distribution table.

  Parameters
  ----------
  td: pandas DataFrame of t distribution table.
  t: target t-value.

  Returns
  -------
  tuple
    degree of freedom(string), alpha(string), and closest t-value to target(number)

  '''
  found = find_number_values(td, t, True)
  if len(found) != 1:
    print('対象が1件ではありません。最初のものを使用します。')
    print(found)
  v = found[0][0]
  alpha = found[0][1]
  return v, alpha, td.at[v, alpha]

In [ ]:
#@title #### t分布表で指定された $t$ 値に最も近い $t$ 値の行（自由度）と列（$\alpha$）を強調表示する関数
#@markdown 関数名：`highlight_t_in_td_table`
#@markdown * 引数：`td`, `t`
#@markdown * 戻り値：`Styler`
#@markdown * 依存関数：`find_alpha_n_from_t_in_td_table`, `highlight_subject`
def highlight_t_in_td_table(td, t):
  '''Hightlight closest t-value to target in t distribution table.

  Parameters
  ----------
  td: pandas DataFrame of t distribution table.
  t: t-value of t distribution.

  Returns
  -------
  Styler
    pandas Styler object to be highlighted

  '''
  p = find_alpha_n_from_t_in_td_table(td, t)
  return highlight_subject(td, row=p[0], col=p[1])

In [ ]:
#@title #### t分布表から指定された自由度と $\alpha$ で $t$ 値を取得する関数
#@markdown 関数名：`find_t_in_td_table`
#@markdown * 引数：`td`, `v`, `alpha`
#@markdown * 戻り値：`float`
def find_t_in_td_table(td, v, alpha):
  '''Find t-value from degrees of freedom and alpha in t distribution table.

  Parameters
  ----------
  td: pandas DataFrame of t distribution table.
  v: degrees of freedom.
  alpha: upper n-parcent point.

  Returns
  -------
  float
    t-value(float)

  '''
  return td.at[f'{v:d}', f'{alpha:.3f}']


In [ ]:
#@title #### 信頼係数から $\alpha$ に変換する関数
#@markdown 関数名：`cf2alpha`
#@markdown * 引数：`cf`
#@markdown * 戻り値：`float`
#@markdown
#@markdown $\alpha = 1 - \dfrac{CF}{100}$
def cf2alpha(cf):
  '''Convert confidence factor to alpha value.

  Parameters
  ----------
  cf: confidence factor

  Returns
  -------
  float
    alpha value
  '''
  return 1 - (cf / 100)

In [ ]:
#@title #### t分布表から指定された自由度と信頼係数で $t$ 値を取得する関数
#@markdown 関数名：`find_t_from_cf_in_td_table`
#@markdown * 引数：`td`, `v`, `cf=95`
#@markdown * 戻り値：`tuple<float, float>`
#@markdown * 依存関数：`cf2alpha`, `find_t_in_td_table`
def find_t_from_cf_in_td_table(td, v, cf=95):
  '''Find t-value from degrees of freedom and alpha in t distribution table.

  Parameters
  ----------
  td: pandas DataFrame of t distribution table.
  v: degrees of freedom.
  cf: Target confidence factor. Default value is 95.

  Returns
  -------
  tuple
    t-value(float) and alpha

  '''
  alpha = cf2alpha(cf) / 2
  return find_t_in_td_table(td, v, alpha), alpha


In [ ]:
#@title #### 信頼区間（t分布）の計算 { display-mode: "form" }
#@markdown 関数名：`calc_conf_interval_td`
#@markdown * 引数：`xbar`, `s2`, `n`, `cf=95`,`td=generate_td_table()`
#@markdown * 戻り値： `tuple<float, float, float, float>`
#@markdown * 依存関数：`generate_td_table`, `find_t_from_cf_in_td_table`
def calc_conf_interval_td(xbar, s2, n, cf=95, td=generate_td_table()):
  '''Calculate confidence interval of t distribution.

  Parameters
  ----------
  xbar: Sample mean.
  s2: unbiased estimate of variance.
  n: Sample size.
  cf: Target confidence factor. Default value is 95.
  td: pandas DataFrame of t distribution table.
    Default value is generate_td_table function execution result

  Returns
  -------
  tuple
    lower bound, upper bound, t-value, and standard error
  '''
  import math
  t = find_t_from_cf_in_td_table(td, n - 1, cf)[0]
  se = math.sqrt(s2 / n)
  bin = t * se
  return xbar - bin, xbar + bin, t, se


In [ ]:
#@title #### プールした分散の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_pooled_var`
#@markdown * 引数：`s1`, `s2`
#@markdown   * `s1`: `tuple<integer, number>`
#@markdown   * `s2`: `tuple<integer, number>`
#@markdown * 戻り値： `float`
#@markdown
#@markdown $s^2 = \dfrac{(n_1 - 1) \times s_1^2 + (n_2 - 1) \times s_2^2}{n_1 + n_2 - 2}$
def calc_pooled_var(s1, s2):
  '''Calculate pooled variance.

  Parameters
  ----------
  s1: tuple of sample size and unbiased estimate of variance of first sample.
  s2: tuple of sample size and unbiased estimate of variance of second sample.

  Returns
  -------
  float
    pooled variance.
  '''
  return ((s1[0] - 1) * s1[1] + (s2[0] - 1) * s2[1]) / (s1[0] + s2[0] - 2)

In [ ]:
#@title #### 対応のないデータの差の信頼区間の計算 { display-mode: "form" }
#@markdown 関数名：`calc_conf_interval_unpaired`
#@markdown * 引数：`p1`, `p2`, `cf=95`,`td=generate_td_table()`
#@markdown * 戻り値： `tuple<float, float, float, float>`
#@markdown * 依存関数：`generate_td_table`, `find_t_from_cf_in_td_table`, `calc_pooled_var`
def calc_conf_interval_unpaired(p1, p2, cf=95, td=generate_td_table()):
  '''Calculate confidence intervals for differences in unpaired data.

  Parameters
  ----------
  p1: tuple of first population parameters(sample mean, unbiased estimate of variance, sample size).
  p2: tuple of second population parameters(sample mean, unbiased estimate of variance, sample size).
  cf: Target confidence factor. Default value is 95.
  td: pandas DataFrame of t distribution table.
    Default value is generate_td_table function execution result

  Returns
  -------
  tuple
    lower bound, upper bound, t-value, and pooled variance.
  '''
  import math
  v = p1[2] + p2[2] - 2
  t = find_t_from_cf_in_td_table(td, v, cf)[0]
  s2_pool = calc_pooled_var((p1[2], p1[1]), (p2[2], p2[1]))
  xbar = p1[0] - p2[0]
  bin = t * math.sqrt(s2_pool * (p1[2] ** -1 + p2[2] ** -1))
  return xbar - bin, xbar + bin, t, s2_pool


In [ ]:
#@title #### 統計量 $t$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_t`
#@markdown * 引数：`xbar`, `mu`, `s2`, `n`
#@markdown * 戻り値：`float`

#@markdown $t = \dfrac{\bar{x} - \mu}{\sqrt{\dfrac{s^2}{n}}}$
def calc_t(xbar, mu, s2, n):
  '''Calculate t-value.

  Parameters
  ----------
  xbar: mean value of data.
  mu: population mean.
  s2: unbiased estimate of variance.
  n: sample size.

  Returns
  -------
  float
    t-value
  '''
  import math
  return (xbar - mu) / math.sqrt(s2 / n)


In [ ]:
#@title #### 対応のない2標本の統計量 $t$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_t_unpaired`
#@markdown * 引数：`s1`, `s2`
#@markdown   * `s1`: `tuple<number, integer, number>`
#@markdown   * `s2`: `tuple<number, integer, number>`
#@markdown * 戻り値： `tuple<float, float>`
#@markdown * 依存関数：`calc_pooled_var`

#@markdown $t = \dfrac{\bar{x}_1 - \bar{x}_2}{\sqrt{s^2 \left(\dfrac{1}{n_1} + \dfrac{1}{n_2}\right)}}$
def calc_t_unpaired(s1, s2):
  '''Calculate t-value of unpaired samples.
  Parameters
  ----------
  s1: tuple of sample mean, sample size, and unbiased estimate of variance of first sample.
  s2: tuple of sample mean, sample size, and unbiased estimate of variance of second sample.

  Returns
  -------
  tuple
    t-value and pooled variance.
  '''
  import math
  pooled_var = calc_pooled_var(s1[1:], s2[1:])
  return (s1[0] - s2[0]) * math.sqrt(pooled_var * (s1[1]**-1 + s2[1]**-1))**-1, pooled_var


In [ ]:
# @title #### 無相関の統計量 $t$ の計算 { vertical-output: true, display-mode: "form" }
# @markdown 関数名：`calc_t_uncorrelated`
# @markdown * 引数：`n`, `r`
# @markdown * 戻り値： `float`

# @markdown $t = \dfrac{|r| \sqrt{n - 2}}{\sqrt{1 - r^2}}$
def calc_t_uncorrelated(n, r):
  '''Calculate t-value of uncorrelated data.

  Parameters
  ----------
  n: sample size.
  r: correlation coefficient.

  Returns
  -------
  float
    t-value of uncorrelated data.
  '''
  import math
  return (abs(r) * math.sqrt(n - 2)) / math.sqrt(1 - r**2)

In [ ]:
# @title #### 分散分析における各水準の母平均の信頼区間の計算 { vertical-output: true, display-mode: "form" }
# @markdown 関数名：`calc_confidence_interval_avt_pm`
# @markdown * 引数：`xbar`, `n`, `dfe`, `ve`, `cf=95`, `td=generate_td_table()`
# @markdown * 戻り値： `tuple<float, float, float, float>`
# @markdown * 依存関数：`generate_td_table`, `cf2alpha`, `find_t_in_td_table`
# @markdown
# @markdown ※計算は同じだが、一元配置・二元配置（交互作用有意）・二元配置（交互作用非有意）で
# @markdown 引数`xbar`と`n`の水準が異なる
def calc_confidence_interval_avt_pm(xbar, n, dfe, ve, cf=95, td=generate_td_table()):
  '''Calculate confidence interval of population mean for each level in analysis of variance.
  Parameters
  ----------
  xbar: sample mean of the level.
  n: sample size of the level.
  dfe: degrees of freedom of residual error.
  ve: mean square of residual error.
  cf: Target confidence factor. Default value is 95.
  td: pandas DataFrame of t distribution table.
    Default value is the result of the generate_td_table function.

  Returns
  -------
  tuple
    lower bound, upper bound, t-value, and alpha.
  '''
  import math
  alpha = cf2alpha(cf) / 2
  t = find_t_in_td_table(td, dfe, alpha)
  bin = t * math.sqrt(ve / n)
  return xbar - bin, xbar + bin, t, alpha


# 利用例

In [ ]:
# @title #### 利用例：`generate_td_table`  { display-mode: "both" }
# @markdown t分布表の表示
td = generate_td_table()
td

,0.250,0.100,0.050,0.025,0.010,0.005
1,1.0000,3.0777,6.3138,12.7062,31.8205,63.6567
2,0.8165,1.8856,2.9200,4.3027,6.9646,9.9248
3,0.7649,1.6377,2.3534,3.1824,4.5407,5.8409
4,0.7407,1.5332,2.1318,2.7764,3.7469,4.6041
5,0.7267,1.4759,2.0150,2.5706,3.3649,4.0321
6,0.7176,1.4398,1.9432,2.4469,3.1427,3.7074
7,0.7111,1.4149,1.8946,2.3646,2.9980,3.4995
8,0.7064,1.3968,1.8595,2.3060,2.8965,3.3554
9,0.7027,1.3830,1.8331,2.2622,2.8214,3.2498
10,0.6998,1.3722,1.8125,2.2281,2.7638,3.1693


In [ ]:
# @title #### 利用例：`generate_td_table` { run: "auto", display-mode: "both" }
# @markdown t分布表の表示（上側$\alpha\%$点および自由度$v$を指定）
# @markdown
# @markdown $\alpha$ と自由度のリスト
alpha = [0.25, 0.1, 0.05, 0.025, 0.001] #@param {type:"raw"}
v = [1, 2, 3, 4, 5, 10, 11, 12, 14, 15, 20, 30, 40, 50, 100, 150, 200] #@param {type:"raw"}

generate_td_table(alpha=alpha, v=v)

,0.250,0.100,0.050,0.025,0.001
1,1.0000,3.0777,6.3138,12.7062,318.3088
2,0.8165,1.8856,2.9200,4.3027,22.3271
3,0.7649,1.6377,2.3534,3.1824,10.2145
4,0.7407,1.5332,2.1318,2.7764,7.1732
5,0.7267,1.4759,2.0150,2.5706,5.8934
10,0.6998,1.3722,1.8125,2.2281,4.1437
11,0.6974,1.3634,1.7959,2.2010,4.0247
12,0.6955,1.3562,1.7823,2.1788,3.9296
14,0.6924,1.3450,1.7613,2.1448,3.7874
15,0.6912,1.3406,1.7531,2.1314,3.7328


In [ ]:
# @title #### 利用例：`highlight_subject` { run: "auto", display-mode: "both" }
# @markdown t分布表の指定された自由度と $\alpha$ を強調表示
# @markdown
# @markdown 自由度と$\alpha$
v = 9 #@param {type:"integer"}
alpha = 0.025 #@param {type:"number"}
highlight_subject(td.head(15), f'{v:d}', f'{alpha:.3f}')

,0.250,0.100,0.050,0.025,0.010,0.005
1,1.000000,3.077700,6.313800,12.706200,31.820500,63.656700
2,0.816500,1.885600,2.920000,4.302700,6.964600,9.924800
3,0.764900,1.637700,2.353400,3.182400,4.540700,5.840900
4,0.740700,1.533200,2.131800,2.776400,3.746900,4.604100
5,0.726700,1.475900,2.015000,2.570600,3.364900,4.032100
6,0.717600,1.439800,1.943200,2.446900,3.142700,3.707400
7,0.711100,1.414900,1.894600,2.364600,2.998000,3.499500
8,0.706400,1.396800,1.859500,2.306000,2.896500,3.355400
9,0.702700,1.383000,1.833100,2.262200,2.821400,3.249800
10,0.699800,1.372200,1.812500,2.228100,2.763800,3.169300


In [ ]:
# @title #### 利用例：`find_alpha_n_from_t_in_td_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown t分布表から指定された $t$ 値に対する自由度と $\alpha$ を検索
t = 2.45 #@param {type:"number"}
find_alpha_n_from_t_in_td_table(td, t)

('6', '0.025', 2.4469)

In [ ]:
# @title #### 利用例：`highlight_t_in_td_table` { run: "auto", display-mode: "both" }
# @markdown t分布表の指定された $t$ 値を強調表示
t = 2.45 #@param {type:"number"}
highlight_t_in_td_table(td.head(15), t)

,0.250,0.100,0.050,0.025,0.010,0.005
1,1.000000,3.077700,6.313800,12.706200,31.820500,63.656700
2,0.816500,1.885600,2.920000,4.302700,6.964600,9.924800
3,0.764900,1.637700,2.353400,3.182400,4.540700,5.840900
4,0.740700,1.533200,2.131800,2.776400,3.746900,4.604100
5,0.726700,1.475900,2.015000,2.570600,3.364900,4.032100
6,0.717600,1.439800,1.943200,2.446900,3.142700,3.707400
7,0.711100,1.414900,1.894600,2.364600,2.998000,3.499500
8,0.706400,1.396800,1.859500,2.306000,2.896500,3.355400
9,0.702700,1.383000,1.833100,2.262200,2.821400,3.249800
10,0.699800,1.372200,1.812500,2.228100,2.763800,3.169300


In [ ]:
# @title #### 利用例：`find_t_in_td_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown t分布表から指定された自由度と $\alpha$ に対する $t$ 値を検索
# @markdown
# @markdown 自由度と$\alpha$
v = 4 #@param {type:"integer"}
alpha = 0.025 #@param {type:"number"}
find_t_in_td_table(td, v, alpha)

2.7764

In [ ]:
# @title #### 利用例：`find_t_from_cf_in_td_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown t分布表から指定された信頼係数に対する $t$ 値を検索
# @markdown
# @markdown 自由度と信頼係数
v = 4 #@param {type:"integer"}
cf = 95 #@param ["90", "95", "99"] {type:"raw"}
find_t_from_cf_in_td_table(td, v, cf)

(2.7764, 0.025000000000000022)

In [ ]:
# @title #### 利用例：`calc_conf_interval_td` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown t分布の信頼区間の計算
# @markdown
# @markdown 標本平均、不偏分散、サンプルサイズ、信頼係数
xbar = 100.03 #@param {type:"number"}
s2 = 2.22 #@param {type:"number"}
n = 10 #@param {type:"integer"}
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}

ci = calc_conf_interval_td(xbar, s2, n, cf)
print('lower:', round_up(2)(ci[0]), ', upper:', round_up(2)(ci[1]), ', t:', round_up(5)(ci[2]))

lower: 98.96 , upper: 101.1 , t: 2.2622


In [ ]:
# @title #### 利用例：`calc_conf_interval_unpaired` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 対応のない母集団に対する信頼区間の計算
# @markdown
# @markdown 母集団１の統計量（標本平均、不偏分散、サンプルサイズ）
xbar_1 = 76 #@param {type:"number"}
s2_1 = 242.5 #@param {type:"number"}
n_1 = 5 #@param {type:"integer"}
# @markdown 母集団２の統計量（標本平均、不偏分散、サンプルサイズ）
xbar_2 = 78.75 #@param {type:"number"}
s2_2 = 272.9 #@param {type:"number"}
n_2 = 4 #@param {type:"integer"}
# @markdown 信頼係数
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}

ci = calc_conf_interval_unpaired((xbar_1, s2_1, n_1), (xbar_2, s2_2, n_2), cf)
print('lower:', round_up(2)(ci[0]), ', upper:', round_up(2)(ci[1]),
      ', t-value:', round_up(5)(ci[2]), ', pooled variance:', round_up(2)(ci[3]))

lower: -28.11 , upper: 22.61 , t-value: 2.3646 , pooled variance: 255.53


In [ ]:
# @title #### 利用例：`calc_pooled_var` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown プールされた分散の計算
# @markdown
# @markdown 母集団１の統計量（サンプルサイズ、不偏分散）
n_1 = 5 #@param {type:"integer"}
s2_1 = 242.5 #@param {type:"number"}
# @markdown 母集団２の統計量（サンプルサイズ、不偏分散）
n_2 = 4 #@param {type:"integer"}
s2_2 = 272.9 #@param {type:"number"}
calc_pooled_var((n_1, s2_1), (n_2, s2_2))

255.5285714285714

In [ ]:
# @title #### 利用例：`calc_t` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown $t$ 値の計算
# @markdown
# @markdown 標本平均、母平均、不偏分散、サンプルサイズ
xbar = 76 #@param {type:"number"}
mu = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}
s2 = 242.5 #@param {type:"number"}
n = 5 #@param {type:"integer"}
calc_t(xbar, mu, s2, n)

-2.7282410027474047

In [ ]:
# @title #### 利用例：`calc_t_unpaired` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 対応のない母集団の $t$ 値の計算
# @markdown
# @markdown 母集団１の統計量（標本平均、不偏分散、サンプルサイズ）
xbar_1 = 75 #@param {type:"number"}
s2_1 = 25 #@param {type:"number"}
n_1 = 30 #@param {type:"integer"}
# @markdown 母集団２の統計量（標本平均、不偏分散、サンプルサイズ）
xbar_2 = 70 #@param {type:"number"}
s2_2 = 64 #@param {type:"number"}
n_2 = 32 #@param {type:"integer"}
calc_t_unpaired((xbar_1, n_1, s2_1), (xbar_2, n_2, s2_2))

(2.928066250005228, 45.15)

In [ ]:
# @title #### 利用例：`calc_t_uncorrelated` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 相関のない $t$ 値の計算
# @markdown
# @markdown サンプルサイズ、標本の相関係数
n = 30 #@param {type:"integer"}
r = -0.30 #@param {type:"number"}
calc_t_uncorrelated(n, r)

1.6641005886756874

In [ ]:
# @title #### 利用例：`calc_confidence_interval_avt_pm` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 分散分析における各水準の母平均の信頼区間の計算
# @markdown
# @markdown 水準 $j$ の標本平均、サンプルサイズ
xbar = 9.3 #@param {type:"number"}
n = 7 #@param {type:"integer"}
# @markdown 残差の自由度、平均平方
dfe = 40 #@param {type:"integer"}
ve = 199.42 #@param {type:"number"}
# @markdown 信頼係数
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}

ci = calc_confidence_interval_avt_pm(xbar, n, dfe, ve, cf)
print('lower:', round_up(2)(ci[0]), ', upper:', round_up(2)(ci[1]),
      ', t-value:', round_up(5)(ci[2]), ', alpha:', round_up(5)(ci[3]))

lower: -1.49 , upper: 20.09 , t-value: 2.0211 , alpha: 0.025
